In [1]:
import sys

sys.path.insert(0, "../")

Azure = True

import json
from torch.utils.data import DataLoader

from dataLoader.dataLoaderMLM import CoercionRiskDataset, process_data_CoercionRisk
from dataLoader.build_vocab import load_vocab, build_vocab

In [2]:
if Azure:
    file_config = {
        "vocab": "../dataloader/vocab.txt",  # vocabulary idx2token, token2idx
        "data_train": "../../EHR_data/data/fine_tune_train_set.json",  # formated data
        "data_val": "../../EHR_data/CoercionData_val.json",  # formated data
        "model_path": "model/model1/",  # where to save model
        "model_name": "test",  # model name
        "file_name": "log",  # log path
        "use_cuda": True,
        "device$": "cuda:0",
    }
else:
    file_config = {
        "vocab": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/vocab.txt",  # vocabulary idx2token, token2idx
        "data_train": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/syntheticData.json",
        "data_val": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/syntheticData_val.json",
        "model_path": "model/model1/",  # where to save model
        "model_name": "test",  # model name
        "file_name": "log.txt",  # log path
        "use_cuda": False,
        "device": "cpu",
    }

In [3]:
with open(file_config["data_train"]) as f:
    data_json = json.load(f)
print(next(iter(data_json.items())))
vocab_list, word_to_idx = build_vocab(data_json, Azure=Azure)

('2445376', {'Sex': 'Mand', 'BirthDate': '1973-09-04', 'DeathDate': None, 'Events': [{'EncounterKey': 85470028, 'Time': '2020-02-12T10:30:00', 'Type': 'Telefonkontakt'}, {'EncounterKey': 77322745, 'Time': '2019-10-22T09:30:00', 'Type': 'Telefonkontakt'}, {'EncounterKey': 74074745, 'Time': '2019-09-12T09:30:00', 'Type': 'Telefonkontakt'}, {'EncounterKey': 95209667, 'Time': '2020-07-07T09:30:00', 'Type': 'Telefonkontakt'}, {'EncounterKey': 88514442, 'Time': '2020-09-09T13:00:00', 'Type': 'Aftale'}, {'EncounterKey': 68215069, 'Time': '2019-10-08T10:30:00', 'Type': 'Aftale'}, {'EncounterKey': 48740486, 'Time': '2018-10-16T11:15:00', 'Type': 'Aftale'}, {'EncounterKey': 8850285, 'Time': '2017-07-13T10:00:00', 'Type': 'Ambulant besøg'}, {'EncounterKey': 62579869, 'Time': '2019-04-09T11:15:00', 'Type': 'Ambulant besøg'}, {'EncounterKey': 1014067, 'Time': '2018-08-15T11:00:00', 'Type': 'Aftale'}, {'EncounterKey': 39268646, 'Time': '2018-04-06T11:00:00', 'Type': 'Aftale'}, {'EncounterKey': 11870

Vocab: 100%|██████████| 5818/5818 [00:00<00:00, 24701.75it/s]


In [4]:
# Data loader
data = process_data_CoercionRisk(
    data_json, vocab_list, word_to_idx, mask_prob=0.20, Azure=Azure
)
sample_data = CoercionRiskDataset(data)
sample = next(iter(sample_data))

 17%|█▋        | 1001/5818 [00:20<01:40, 47.81it/s]


In [5]:
import itertools

# sample = next(iter(data.items()))

# Replace 'N' with the desired item number you want to access (in this case, 10)
item_number = 10100

# Using itertools.islice to get the Nth item (key-value pair) from the dictionary
# sample = next(itertools.islice(data.items(), item_number))

count  = 0
for sample in data.items():
    index = sample[1]['sample_index']
    if sample[1]['classification_labels'] == 1:
        print('patient',sample[1]['patient'])
        print('date',sample[1]['dates'][index-5:index+5])
        print('age',sample[1]['age'][index-5:index+5])
        print('codes',sample[1]['codes'][index-5:index+5])
        print('position',sample[1]['position'][index-5:index+5])
        print('segment',sample[1]['segment'][index-5:index+5])
        print('classification_labels',sample[1]['classification_labels'])
        print('original_code_sequence',sample[1]['original_code_sequence'][index-5:index+5])
        print('sample_index',sample[1]['sample_index'])
        print('input_sequence',sample[1]['input_sequence'][index-5:index+5])
        print('attention_mask',sample[1]['attention_mask'][index-5:index+5])
        break
    count +=1




date [42723, 42877, 42877, 42915, 0, 0, 0, 0, 0, 0]
age [57.916666666666664, 58.333333333333336, 58.333333333333336, 58.5, 0, 0, 0, 0, 0, 0]
codes ['Ambulant besøg', '<SEP>', 'Behandlingskontakt', '3 Akut ambulant', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
position [1, 1, 2, 2, 0, 0, 0, 0, 0, 0]
segment [2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
classification_labels 1
original_code_sequence ['Ambulant besøg', '<SEP>', 'Behandlingskontakt', '3 Akut ambulant', '<SEP>', 'coercion_3_start', 'coercion_3_start', 'coercion_1_start', 'coercion_3_start', 'DF21_29']
sample_index 6
input_sequence [19, 2, 51, 63, 3, 3, 3, 3, 3, 3]
attention_mask [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
